In [2]:
# always check your python version
!python --version

Python 3.11.9


In [3]:
# AWS SDK for Python, -q flag is for quiet
%pip install -q boto3

Note: you may need to restart the kernel to use updated packages.


In [4]:
# always check your boto3 version
%pip show boto3

Name: boto3
Version: 1.34.131
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.11/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: amazon-q-developer-jupyterlab-ext, amazon-sagemaker-sql-editor, amazon-sagemaker-sql-execution, amazon_sagemaker_jupyter_ai_q_developer, autogluon.common, autogluon.core, autogluon.multimodal, aws-glue-sessions, langchain-aws, PyAthena, redshift_connector, sagemaker, sagemaker-mlflow, sagemaker-studio-analytics-extension, sagemaker-studio-sparkmagic-lib
Note: you may need to restart the kernel to use updated packages.


In [6]:
import boto3
client = boto3.client("bedrock-runtime")

In [7]:
# This is a prompt optimized for Titan.
# https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Text+Prompt+Engineering+Guidelines.pdf
prompt_titan = """
User: 
Sulfuric acid reacts with sodium chloride, and gives {{chemical1}} and {{chemical2}}:
Bot:
"""

# This is a prompt optimized for Claude 
# https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/overview
prompt_claude = """
Human: 
Sulfuric acid reacts with sodium chloride, and gives <chemical1>_____</chemical1> and <chemical2>_____</chemical2>:
Assistant:
"""

In [8]:
# Before you can invoke the model you need to enable model access in Amazon Bedrock

import json
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/invoke_model.html
# https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags
resp = client.invoke_model(
  modelId='amazon.titan-text-express-v1',
  body=json.dumps({
    "inputText": prompt_titan
  })
)
body = json.loads(resp.get("body").read())
print(body['results'][0]['outputText'])

The chemical reaction between sulfuric acid and sodium chloride is a single replacement reaction. Here is the word equation for the reaction:
2HCl + Na2SO4 → 2NaCl + H2SO4
The chemical reaction between sulfuric acid and sodium chloride is a double replacement reaction. Here is the word equation for the reaction:
Na2SO4 + 2HCl → NaCl + H2SO4
The chemical reaction between sulfuric acid and sodium chloride is a precipitation reaction. Here is the word equation for the reaction:
NaCl + H2SO4 


In [9]:
# Notice how Claude 2 use text completion
#resp = client.invoke_model(
#    modelId='anthropic.claude-v2',
#    contentType='application/json',
#    accept='application/json',
#    body=json.dumps({
#        "prompt": prompt,
#        "max_tokens_to_sample": 200,
#        "temperature": 1.0
#    })
#)
#body = json.loads(resp.get("body").read())
#print(body['completion'])


# Notice how Claude 3 uses the Message API
# https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html
# https://docs.anthropic.com/en/api/messages
resp = client.invoke_model(
    modelId='anthropic.claude-3-haiku-20240307-v1:0',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "messages": [{"role": "user", "content": [{"type": "text", "text": prompt_claude}]}],
        "max_tokens": 200,
        "temperature": 1.0,
        "anthropic_version": "bedrock-2023-05-31"
    })
)
body = json.loads(resp.get("body").read())
print(body['content'][0]['text'])

The reaction between sulfuric acid (H2SO4) and sodium chloride (NaCl) produces the following products:

<chemical1>Hydrogen chloride (HCl)</chemical1>
<chemical2>Sodium sulfate (Na2SO4)</chemical2>

The balanced chemical equation for this reaction is:

2 NaCl + H2SO4 → 2 HCl + Na2SO4

In this reaction, the sulfuric acid (H2SO4) reacts with the sodium chloride (NaCl) to produce hydrogen chloride (HCl) and sodium sulfate (Na2SO4).


In [10]:
# Lets create a helper.py and a text file loader
from helper import load_text_file
prompt = load_text_file("prompts/zero-shot.txt")
print(prompt)

Human: 
Sulfuric acid reacts with sodium chloride, and gives <chemical1>_____</chemical1> and <chemical2>_____</chemical2>:
Assistant:


In [11]:
resp = client.invoke_model(
    modelId='anthropic.claude-3-haiku-20240307-v1:0',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
        "max_tokens": 200,
        "temperature": 1.0,
        "anthropic_version": "bedrock-2023-05-31"
    })
)
body = json.loads(resp.get("body").read())
print(body['content'][0]['text'])

The reaction between sulfuric acid (H2SO4) and sodium chloride (NaCl) produces the following products:

<chemical1>Hydrogen chloride (HCl)</chemical1>
<chemical2>Sodium sulfate (Na2SO4)</chemical2>

The balanced chemical equation for this reaction is:

2 NaCl + H2SO4 → 2 HCl + Na2SO4

In this reaction, the sulfuric acid (H2SO4) reacts with the sodium chloride (NaCl) to produce hydrogen chloride (HCl) and sodium sulfate (Na2SO4).
